In [1]:
import scanpy as sc
data_dir = '../../dataset/Human_Great_Apes_filtered.h5ad'
adata = sc.read_h5ad(data_dir)
print(adata.X)
sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 753825891 stored elements and shape (156285, 17413)>
  Coords	Values
  (0, 12862)	0.5665589570999146
  (0, 3064)	0.5665589570999146
  (0, 13278)	3.3999741077423096
  (0, 813)	2.833569049835205
  (0, 13496)	4.101833820343018
  (0, 5722)	2.5798444747924805
  (0, 15309)	7.879579067230225
  (0, 2014)	0.5665589570999146
  (0, 7585)	5.2227888107299805
  (0, 6840)	5.497739791870117
  (0, 12721)	3.967062473297119
  (0, 15364)	1.8461450338363647
  (0, 8240)	1.5708974599838257
  (0, 11878)	2.2390189170837402
  (0, 8802)	5.875441551208496
  (0, 4476)	4.394822597503662
  (0, 10574)	0.5665589570999146
  (0, 16860)	6.168292999267578
  (0, 2182)	5.933631896972656
  (0, 8212)	2.520345449447632
  (0, 7982)	5.331713676452637
  (0, 2798)	5.730631351470947
  (0, 6747)	0.9259976744651794
  (0, 2034)	6.668968677520752
  (0, 9759)	3.3205926418304443
  :	:
  (156284, 2496)	3.841771125793457
  (156284, 12674)	3.841771125793457
  (156284, 3173)	4.524

In [2]:
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPAutoEncoder
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/RandomMask.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]
units_decoder = [256, 256, 512, 512]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPAutoEncoder.load_from_checkpoint(
    ckpt_path,
    gene_dim=19331,  # 根据你的数据调整
    batch_size=2048,  # 根据你的需要调整
    units_encoder=units_encoder, 
    units_decoder=units_decoder,
    masking_strategy="random",  # 假设模型使用了随机掩码
    masking_rate=0.5,  # 根据需要调整
)

# 使用 GPU 进行评估（如果可用）
estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPAutoEncoder(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (encoder): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=256, out_features=256, b

In [3]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [4]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [5]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

feature
OPN1MW    OPN1MW
KY            KY
COL6A6    COL6A6
EVI2A      EVI2A
UBE2D4    UBE2D4
           ...  
SDF2        SDF2
OSR1        OSR1
SOWAHB    SOWAHB
CHCHD3    CHCHD3
ACTR8      ACTR8
Name: gene_name, Length: 17413, dtype: category
Categories (29813, object): ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', ..., 'ZYG11AP1', 'ZYG11B', 'ZZEF1', 'ZZZ3']

In [6]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)


In [7]:
existing_genes = adata.var['gene_name']
existing_genes

feature
OPN1MW    OPN1MW
KY            KY
COL6A6    COL6A6
EVI2A      EVI2A
UBE2D4    UBE2D4
           ...  
SDF2        SDF2
OSR1        OSR1
SOWAHB    SOWAHB
CHCHD3    CHCHD3
ACTR8      ACTR8
Name: gene_name, Length: 17413, dtype: category
Categories (29813, object): ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', ..., 'ZYG11AP1', 'ZYG11B', 'ZZEF1', 'ZZZ3']

In [8]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 17413
匹配的基因列表: {'slc25a43', 'nasp', 'slc30a2', 'baz1a', 'plxnc1', 'tm9sf2', 'polg2', 'gpx7', 'sfxn2', 'serping1', 'champ1', 'tma16', 'galnt4', 'melk', 'ffar3', 'tspan2', 'c1d', 'hydin', 'akap8l', 'zfp3', 'hebp2', 'racgap1', 'lsm3', 'gar1', 'uvssa', 'chfr', 'gja9', 'hrk', 'sik1', 'hspb2-c11orf52', 'rbmy1d', 'pgap3', 'pik3r4', 'stmn4', 'rlbp1', 'pex11g', 'lpin1', 'znf561', 'ddx3x', 'slc22a3', 'ceacam6', 'atp6v1b2', 'aldh3a2', 'srxn1', 'cox6a1', 'ddtl', 'nudt2', 'nob1', 'plcb4', 'spock2', 'insig2', 'smo', 'mrps34', 'rpf2', 'stk35', 'ccl1', 'col14a1', 'pak1', 'adcy5', 'macrod2', 'rbm12', 'snx21', 'sh2d3a', 'amy1b', 'ppp2r2d', 'tnfsf11', 'znf429', 'ikbke', 'grik2', 'nfya', 'ninj1', 'clec14a', 'rnf181', 'prox2', 'ncr2', 'eno1', 'jpt2', 'ocm2', 'gata5', 'socs4', 'eif4h', 'gxylt1', 'brs3', 'alg1', 'or13c8', 'gabrr2', 'vegfd', 'cd1a', 'esam', 'arhgef7', 'fgf21', 'ppara', 'pxdnl', 'capn8', 'tm2d3', 'or9q1', 'ankrd40', 'bola3', 'hspa12a', 'cacna1h', 'fancf', 'tcf12', 'or4d6', 'or7c2', 'c

set()

In [9]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [10]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 1918
仅在 all_genes 中存在的基因: {'cenpp', 'slc25a12', 'dhrs4l2', 'htn3', 'lexm', 'c9orf135', 'ly6h', 'arl17b', 'psors1c2', 'slc25a6', 'jhy', 'zp2', 'slc25a24', 'slc25a26', 'napepld', 'clic1', 'fndc11', 'casp8ap2', 'marf1', 'ppp4r4', 'fam174c', 'magel2', 'pramef26', 'mrps18b', 'c22orf15', 'ppp1r3b', 'or2j2', 'c17orf113', 'or11h6', 'mgam', 'ifnl3', 'ccnb1ip1', 'cox5b', 'vsig10l2', 'pramef13', 'rnf151', 'pmis2', 'padi2', 'kir3dl2', 'armc10', 'cbwd5', 'ly6g6f-ly6g6d', 'c13orf46', 'smim11a', 'gng14', 'tp53tg3f', 'lbhd2', 'ct47a1', 'foxo6', 'ints3', 'rps9', 'eloa3cp', 'ldah', 'ccl4l2', 'tex46', 'mrnip', 'fam189b', 'prss58', 'c11orf95', 'surf6', 'brd2', 'cfap206', 'krtap4-6', 'dux4', 'trpm1', 'radx', 'pcp4', 'adam32', 'rnh1', 'rbfox2', 'ly6g5c', 'mpig6b', 'deaf1', 'or2a5', 'lpcat4', 'nop9', 'usp17l7', 'majin', 'chmp4a', 'bag6', 'kir3dl1', 'npipa1', 'ajm1', 'faap100', 'spata46', 'garre1', 'praf2', 'mmp28', 'tas2r9', 'btn2a1', 'kir2dl3', 'kifc1', 'tmem251', 'prss51', 'mfhas1', 

In [11]:
dense_adata_X = adata.X.toarray()
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = dense_adata_X[:, i]
    else:
        print(f'Gene {gene} not found in all_genes list')


In [12]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42
X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.20, random_state=random_seed)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

estim.model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train).float().to(device)
    X_test_tensor = torch.tensor(X_test).float().to(device)
    train_embeddings = estim.model.encoder(X_train_tensor).detach().cpu().numpy()
    test_embeddings = estim.model.encoder(X_test_tensor).detach().cpu().numpy()


cuda


In [13]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.9943052756182615
Weighted F1 Score: 0.9942745849726079
Macro F1 Score: 0.9804679940423888
Random Guess Accuracy: 0.32775393340845166
                                                                precision    recall  f1-score   support

     L2/3-6 intratelencephalic projecting glutamatergic neuron       1.00      1.00      1.00     17236
L5 extratelencephalic projecting glutamatergic cortical neuron       1.00      0.93      0.96        87
                             L6b glutamatergic cortical neuron       1.00      1.00      1.00       702
                            VIP GABAergic cortical interneuron       0.99      1.00      0.99      2022
                              astrocyte of the cerebral cortex       1.00      0.99      0.99       639
                caudal ganglionic eminence derived interneuron       0.96      0.89      0.92       162
                              cerebral cortex endothelial cell       0.93      1.00      0.96        27
     

In [14]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "Human_Great_Apes_random_mask_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicro_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['Human_Great_Apes'],
    'method': [re.search(r'Human_Great_Apes_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micro_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.980	0.994	0.994
